In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [8]:
secure_df = pd.read_csv("secure.csv")
vulnerable_df = pd.read_csv("vulnerable.csv")

In [11]:
df = pd.concat([secure_df, vulnerable_df], ignore_index=True)
print(len(df))

36671


In [15]:
contract_codes = []
for hash_id in df['hash_id']:
    file_path = f"source/{hash_id}.sol"  
    with open(file_path, 'r', encoding='utf-8') as file:
        contract_codes.append(file.read())

df['contract_code'] = contract_codes

In [25]:
from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

def get_codebert_embedding(code):
    inputs = tokenizer(code, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  

df['code_embedding'] = df['contract_code'].apply(get_codebert_embedding)


ModuleNotFoundError: No module named 'transformers'